## 1. Import Required Libraries

In [2]:
import sys
import pandas as pd
from pathlib import Path

# Add subtask1 directory to path for imports
sys.path.insert(0, str(Path.cwd() / 'subtask1'))

# Import preprocessing modules
from ArbPreBasic import ArabicBasicPreprocessor
from ArbPreAdv import ArabicAdvancedPreprocessor

# Import feature engineering module (reload to get latest changes)
import importlib
import feature_engineering
importlib.reload(feature_engineering)

from feature_engineering import (
    Subtask1FeatureEngineer,
    Subtask2FeatureEngineer,
    Subtask3FeatureEngineer,
    ClassificationDecision,
    CascadingResult,
)

print("✓ All modules imported successfully")

✓ All modules imported successfully


## 2. Configure Pipeline Settings

**Select your preprocessing method and target subtask:**

In [3]:
# ============================================
# USER CONFIGURATION
# ============================================

# PREPROCESSING METHOD: Choose 'basic' or 'advanced'
PREPROCESSING_METHOD = 'basic'  # Options: 'basic', 'advanced'

# TARGET SUBTASK: Choose 1, 2, or 3
TARGET_SUBTASK = 1  # Options: 1 (binary polarization), 2 (multi-label toxicity), 3 (hate-speech severity)

# ============================================
# ADVANCED PREPROCESSING OPTIONS (only used if PREPROCESSING_METHOD = 'advanced')
# ============================================
ADVANCED_CONFIG = {
    'split_proclitics': None,  # Set of proclitic types to split, e.g., {'CONJ', 'PREP'}
    'split_enclitics': {'PRON'},  # Split pronominal enclitics (recommended)
    'keep_definite_article': True,  # Keep 'Al+' attached (recommended)
    'keep_particles': True,  # Keep particles attached (recommended)
    'use_light_stemming': False,  # Apply light stemming
    'use_lemmatization': False,  # Apply lemmatization
    'use_basic_preprocessing': True  # Apply basic preprocessing first
}

print(f"Configuration:")
print(f"  Preprocessing: {PREPROCESSING_METHOD.upper()}")
print(f"  Target Subtask: {TARGET_SUBTASK}")

Configuration:
  Preprocessing: BASIC
  Target Subtask: 1


## 3. Initialize Preprocessing Pipeline

In [4]:
# Initialize the selected preprocessor
if PREPROCESSING_METHOD.lower() == 'basic':
    preprocessor = ArabicBasicPreprocessor()
    print("✓ Initialized ArabicBasicPreprocessor")
    print("  - Character standardization (Alef, Hamza, Yaa variants)")
    print("  - Diacritics removal")
    print("  - Tatweel removal")
    
elif PREPROCESSING_METHOD.lower() == 'advanced':
    preprocessor = ArabicAdvancedPreprocessor(**ADVANCED_CONFIG)
    print("✓ Initialized ArabicAdvancedPreprocessor")
    print("  - All basic preprocessing features")
    print("  - Morphological analysis using CAMeL Tools")
    print("  - Selective segmentation (pronominal enclitics)")
    if ADVANCED_CONFIG['use_lemmatization']:
        print("  - Lemmatization enabled")
    if ADVANCED_CONFIG['use_light_stemming']:
        print("  - Light stemming enabled")
else:
    raise ValueError(f"Invalid preprocessing method: {PREPROCESSING_METHOD}. Choose 'basic' or 'advanced'.")

print()

✓ Initialized ArabicBasicPreprocessor
  - Character standardization (Alef, Hamza, Yaa variants)
  - Diacritics removal
  - Tatweel removal



## 4. Initialize Feature Engineering

In [5]:
# Initialize the selected feature engineer
if TARGET_SUBTASK == 1:
    feature_engineer = Subtask1FeatureEngineer(preprocessor=preprocessor)
    subtask_name = "Binary Polarization"
    prefilter_threshold = 0.30
    
elif TARGET_SUBTASK == 2:
    feature_engineer = Subtask2FeatureEngineer(preprocessor=preprocessor)
    subtask_name = "Multi-label Toxicity"
    prefilter_threshold = 0.25
    
elif TARGET_SUBTASK == 3:
    feature_engineer = Subtask3FeatureEngineer(preprocessor=preprocessor)
    subtask_name = "Hate-Speech Severity"
    prefilter_threshold = 0.20
    
else:
    raise ValueError(f"Invalid subtask: {TARGET_SUBTASK}. Choose 1, 2, or 3.")

print(f"✓ Initialized Subtask {TARGET_SUBTASK} Feature Engineer ({subtask_name})")
print("  - Dialectal normalization")
print("  - Toxic lexicon ratio scoring (Mubarak et al., 2017)")
print(f"  - Prefilter threshold: {prefilter_threshold}")
print()
print("Pipeline ready!")

✓ Initialized Subtask 1 Feature Engineer (Binary Polarization)
  - Dialectal normalization
  - Toxic lexicon ratio scoring (Mubarak et al., 2017)
  - Prefilter threshold: 0.3

Pipeline ready!


## 5. Test Pipeline on Sample Texts

## 5b. Cascading Classifier - Batch Processing Demo

The cascading classifier uses rule-based pre-filtering to bypass deep learning for obvious cases:
- **TOXIC** (high confidence): toxic_ratio > 0.15
- **CLEAN** (moderate confidence): toxic_ratio < 0.02
- **UNCERTAIN**: Falls back to model prediction (~40% of samples)

In [6]:
# Batch cascading classification demo
batch_texts = [
    "كتابهم جميل والمعلمون يدرسون في المدرسة",  # Clean
    "شعب جاهل وحكام فاسقين كلاب",  # Toxic (3 hits)
    "هذا النص عادي",  # Clean
    "انت حمار وغبي",  # Toxic (2 hits in 3 tokens = 0.67)
    "الطقس جميل اليوم",  # Clean
    "يا كلب يا خنزير",  # Toxic
    "مرحبا كيف حالك",  # Clean
    "انت منافق وخائن",  # Toxic
]

# Apply batch cascading classifier
results = feature_engineer.cascading_classify_batch(
    batch_texts,
    toxic_threshold=0.15,
    clean_threshold=0.02
)

# Display results
print("BATCH CASCADING CLASSIFICATION RESULTS")
print("="*80)
for i, (text, result) in enumerate(zip(batch_texts, results), 1):
    status = "🔴 TOXIC" if result.decision.value == "TOXIC" else "🟢 CLEAN" if result.decision.value == "CLEAN" else "🟡 UNCERTAIN"
    model_flag = "📊 Model" if result.requires_model else "⚡ Rule"
    print(f"{i}. {status} ({model_flag}) | ratio={result.toxic_ratio:.2f} | hits={result.toxic_hits}")
    print(f"   Text: {text[:50]}...")
    print()

# Get cascade statistics
stats = feature_engineer.get_cascade_stats(results)
print("="*80)
print("CASCADE STATISTICS")
print("="*80)
print(f"Total samples: {stats['total']}")
print(f"Rule-based decisions: {stats['rule_based']} ({stats['rule_based_pct']}%)")
print(f"  - Toxic by rule: {stats['toxic_by_rule']}")
print(f"  - Clean by rule: {stats['clean_by_rule']}")
print(f"Model required: {stats['model_required']} ({stats['model_required_pct']}%)")
print()
print("💡 This means the model only needs to process", 
      f"{stats['model_required_pct']}% of samples!")

BATCH CASCADING CLASSIFICATION RESULTS
1. 🟢 CLEAN (⚡ Rule) | ratio=0.00 | hits=[]
   Text: كتابهم جميل والمعلمون يدرسون في المدرسة...

2. 🔴 TOXIC (⚡ Rule) | ratio=0.60 | hits=['جاهل', 'فاسقين', 'كلاب']
   Text: شعب جاهل وحكام فاسقين كلاب...

3. 🟢 CLEAN (⚡ Rule) | ratio=0.00 | hits=[]
   Text: هذا النص عادي...

4. 🔴 TOXIC (⚡ Rule) | ratio=0.33 | hits=['حمار']
   Text: انت حمار وغبي...

5. 🟢 CLEAN (⚡ Rule) | ratio=0.00 | hits=[]
   Text: الطقس جميل اليوم...

6. 🔴 TOXIC (⚡ Rule) | ratio=0.50 | hits=['كلب', 'خنزير']
   Text: يا كلب يا خنزير...

7. 🟢 CLEAN (⚡ Rule) | ratio=0.00 | hits=[]
   Text: مرحبا كيف حالك...

8. 🔴 TOXIC (⚡ Rule) | ratio=0.33 | hits=['منافق']
   Text: انت منافق وخائن...

CASCADE STATISTICS
Total samples: 8
Rule-based decisions: 8 (100.0%)
  - Toxic by rule: 4
  - Clean by rule: 4
Model required: 0 (0.0%)

💡 This means the model only needs to process 0.0% of samples!


In [7]:
# Sample Arabic texts for testing (includes words from expanded lexicon)
sample_texts = [
    "كتابهم جميل والمعلمون يدرسون في المدرسة",  # Clean text
    "شعب جاهل وحكام فاسقين كلاب امريكي والصهاينة",  # Contains: جاهل, فاسقين, كلاب
    "هذا النص يحتوي على كلمات عادية فقط",  # Clean text
    "انت حمار وغبي ومنافق",  # High toxic ratio: حمار, غبي, منافق
    "بعل امون رد بالك توصل الليبيات ياتونسي يابغل",  # High toxic ratio: حمار, غبي, منافق
]

print("="*80)
print("SAMPLE TEXT PROCESSING WITH CASCADING CLASSIFIER")
print("="*80)

for i, text in enumerate(sample_texts, 1):
    print(f"\n{'='*80}")
    print(f"Sample {i}")
    print(f"{'='*80}")
    print(f"Original text:\n  {text}")
    print()
    
    # Process text through the pipeline
    features = feature_engineer.transform_text(text)
    
    print(f"Preprocessed text:\n  {features['preprocessed_text']}")
    print()
    print(f"Dialect-normalized text:\n  {features['dialect_normalized_text']}")
    print()
    print(f"Tokens: {len(features['tokens'])}")
    print(f"  {features['tokens']}")
    print()
    print(f"Toxic Lexicon Ratio: {features['toxic_lexicon_ratio']:.4f}")
    print(f"Toxic Lexicon Hits: {features['toxic_lexicon_hits']}")
    print()
    
    # Apply cascading classifier
    result = feature_engineer.cascading_classify(
        text,
        toxic_threshold=0.15,
        clean_threshold=0.02
    )
    
    print(f"CASCADING CLASSIFIER RESULT:")
    print(f"  Decision: {result.decision.value}")
    print(f"  Confidence: {result.confidence:.2f}")
    print(f"  Requires Model: {'Yes' if result.requires_model else 'No (rule-based)'}")

print(f"\n{'='*80}")

SAMPLE TEXT PROCESSING WITH CASCADING CLASSIFIER

Sample 1
Original text:
  كتابهم جميل والمعلمون يدرسون في المدرسة

Preprocessed text:
  كتابهم جميل والمعلمون يدرسون في المدرسة

Dialect-normalized text:
  كتابهم جميل والمعلمون يدرسون في المدرسة

Tokens: 6
  ['كتابهم', 'جميل', 'والمعلمون', 'يدرسون', 'في', 'المدرسة']

Toxic Lexicon Ratio: 0.0000
Toxic Lexicon Hits: []

CASCADING CLASSIFIER RESULT:
  Decision: CLEAN
  Confidence: 0.85
  Requires Model: No (rule-based)

Sample 2
Original text:
  شعب جاهل وحكام فاسقين كلاب امريكي والصهاينة

Preprocessed text:
  شعب جاهل وحكام فاسقين كلاب امريكي والصهاينة

Dialect-normalized text:
  شعب جاهل وحكام فاسقين كلاب امريكي والصهاينة

Tokens: 7
  ['شعب', 'جاهل', 'وحكام', 'فاسقين', 'كلاب', 'امريكي', 'والصهاينة']

Toxic Lexicon Ratio: 0.4286
Toxic Lexicon Hits: ['جاهل', 'فاسقين', 'كلاب']

CASCADING CLASSIFIER RESULT:
  Decision: TOXIC
  Confidence: 0.90
  Requires Model: No (rule-based)

Sample 3
Original text:
  هذا النص يحتوي على كلمات عادية فقط

P

## 6. Process DataFrame (Batch Processing)

Apply the pipeline to a pandas DataFrame with multiple texts.

In [8]:
# Create a sample DataFrame
sample_df = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'text': [
        "كتابهم جميل والمعلمون يدرسون في المدرسة",
        "شلون الحال؟ وين رايح اليوم؟",
        "هذا النص يحتوي على كلمات عادية فقط",
        "النص الرابع للاختبار"
    ]
})

print("Original DataFrame:")
print(sample_df)
print()

Original DataFrame:
   id                                     text
0   1  كتابهم جميل والمعلمون يدرسون في المدرسة
1   2              شلون الحال؟ وين رايح اليوم؟
2   3       هذا النص يحتوي على كلمات عادية فقط
3   4                     النص الرابع للاختبار



In [9]:
# Apply feature engineering to DataFrame
processed_df = feature_engineer.add_features(
    sample_df,
    text_col='text',
    normalized_col='text_normalized',
    ratio_col='toxic_ratio',
    hits_col='toxic_hits',
    already_preprocessed=False,
    inplace=False
)

print("Processed DataFrame with features:")
print(processed_df[['id', 'text', 'text_normalized', 'toxic_ratio']].to_string())
print()
print("\nDetailed view of toxic lexicon hits:")
for idx, row in processed_df.iterrows():
    print(f"ID {row['id']}: {row['toxic_hits']}")

Processed DataFrame with features:
   id                                     text                          text_normalized  toxic_ratio
0   1  كتابهم جميل والمعلمون يدرسون في المدرسة  كتابهم جميل والمعلمون يدرسون في المدرسة          0.0
1   2              شلون الحال؟ وين رايح اليوم؟                 كيف الحال أين رايح اليوم          0.0
2   3       هذا النص يحتوي على كلمات عادية فقط       هذا النص يحتوي علي كلمات عادية فقط          0.0
3   4                     النص الرابع للاختبار                     النص الرابع للاختبار          0.0


Detailed view of toxic lexicon hits:
ID 1: []
ID 2: []
ID 3: []
ID 4: []


## 7. Load and Process Real Data

Apply the pipeline to actual training/dev data.

In [10]:
# Define data paths based on selected subtask
data_dir = Path(f'subtask{TARGET_SUBTASK}')
train_file = data_dir / 'train' / 'arb.csv'
dev_file = data_dir / 'dev' / 'arb.csv'

print(f"Data directory: {data_dir}")
print(f"Train file: {train_file}")
print(f"Dev file: {dev_file}")
print()

# Load data if files exist
if train_file.exists():
    train_df = pd.read_csv(train_file)
    print(f"✓ Loaded training data: {len(train_df)} samples")
    print(f"  Columns: {list(train_df.columns)}")
else:
    print(f"⚠ Training file not found: {train_file}")
    train_df = None

if dev_file.exists():
    dev_df = pd.read_csv(dev_file)
    print(f"✓ Loaded dev data: {len(dev_df)} samples")
    print(f"  Columns: {list(dev_df.columns)}")
else:
    print(f"⚠ Dev file not found: {dev_file}")
    dev_df = None

Data directory: subtask1
Train file: subtask1/train/arb.csv
Dev file: subtask1/dev/arb.csv

⚠ Training file not found: subtask1/train/arb.csv
⚠ Dev file not found: subtask1/dev/arb.csv


In [11]:
# Process training data (if loaded)
if train_df is not None:
    print("Processing training data...")
    
    # Determine the text column name (might be 'text' or 'sentence')
    text_col = 'text' if 'text' in train_df.columns else 'sentence'
    
    train_processed = feature_engineer.add_features(
        train_df,
        text_col=text_col,
        normalized_col='text_normalized',
        ratio_col='toxic_ratio',
        hits_col='toxic_hits',
        already_preprocessed=False,
        inplace=False
    )
    
    print(f"✓ Processed {len(train_processed)} training samples")
    print(f"\nFirst 3 samples:")
    display_cols = ['id', text_col, 'text_normalized', 'toxic_ratio']
    display_cols = [c for c in display_cols if c in train_processed.columns]
    print(train_processed[display_cols].head(3).to_string())
    
    print(f"\nToxic ratio statistics:")
    print(train_processed['toxic_ratio'].describe())
else:
    print("No training data to process")
    train_processed = None

No training data to process


In [12]:
# Process dev data (if loaded)
if dev_df is not None:
    print("Processing dev data...")
    
    # Determine the text column name
    text_col = 'text' if 'text' in dev_df.columns else 'sentence'
    
    dev_processed = feature_engineer.add_features(
        dev_df,
        text_col=text_col,
        normalized_col='text_normalized',
        ratio_col='toxic_ratio',
        hits_col='toxic_hits',
        already_preprocessed=False,
        inplace=False
    )
    
    print(f"✓ Processed {len(dev_processed)} dev samples")
    print(f"\nFirst 3 samples:")
    display_cols = ['id', text_col, 'text_normalized', 'toxic_ratio']
    display_cols = [c for c in display_cols if c in dev_processed.columns]
    print(dev_processed[display_cols].head(3).to_string())
    
    print(f"\nToxic ratio statistics:")
    print(dev_processed['toxic_ratio'].describe())
else:
    print("No dev data to process")
    dev_processed = None

No dev data to process


## 8. Save Processed Data (Optional)

In [13]:
# Save processed data to CSV files
SAVE_PROCESSED_DATA = False  # Set to True to save

if SAVE_PROCESSED_DATA:
    output_dir = data_dir / 'processed'
    output_dir.mkdir(exist_ok=True)
    
    method_suffix = PREPROCESSING_METHOD[:3]  # 'bas' or 'adv'
    
    if train_processed is not None:
        output_file = output_dir / f'train_arb_{method_suffix}.csv'
        train_processed.to_csv(output_file, index=False)
        print(f"✓ Saved training data to: {output_file}")
    
    if dev_processed is not None:
        output_file = output_dir / f'dev_arb_{method_suffix}.csv'
        dev_processed.to_csv(output_file, index=False)
        print(f"✓ Saved dev data to: {output_file}")
else:
    print("Data saving disabled. Set SAVE_PROCESSED_DATA = True to save processed files.")

Data saving disabled. Set SAVE_PROCESSED_DATA = True to save processed files.


## 9. Pipeline Summary

In [14]:
print("="*80)
print("PIPELINE SUMMARY")
print("="*80)
print(f"Preprocessing Method: {PREPROCESSING_METHOD.upper()}")
print(f"Target Subtask: {TARGET_SUBTASK} ({subtask_name})")
print(f"Prefilter Threshold: {prefilter_threshold}")
print()
print("Processing Steps:")
print("  1. Text preprocessing (character normalization, diacritics removal)")
if PREPROCESSING_METHOD == 'advanced':
    print("  2. Morphological analysis and segmentation (CAMeL Tools)")
print("  3. Dialectal normalization")
print("  4. Toxic lexicon ratio calculation")
print()
print("Output Features:")
print("  - preprocessed_text: Text after basic/advanced preprocessing")
print("  - text_normalized: Text after dialectal normalization")
print("  - toxic_ratio: Proportion of toxic terms (Mubarak lexicon)")
print("  - toxic_hits: List of detected toxic terms")
print("="*80)

PIPELINE SUMMARY
Preprocessing Method: BASIC
Target Subtask: 1 (Binary Polarization)
Prefilter Threshold: 0.3

Processing Steps:
  1. Text preprocessing (character normalization, diacritics removal)
  3. Dialectal normalization
  4. Toxic lexicon ratio calculation

Output Features:
  - preprocessed_text: Text after basic/advanced preprocessing
  - text_normalized: Text after dialectal normalization
  - toxic_ratio: Proportion of toxic terms (Mubarak lexicon)
  - toxic_hits: List of detected toxic terms
